In [22]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
BASE_DATA_PATH = '../datasets/transaction_bank_data.pqt'

In [24]:
import dbscanmethod
import graphsmethod as graphmethod
import matrixmethod
from yousi import DetectRecurrencyII

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [25]:
df = pd.read_parquet(BASE_DATA_PATH)

In [26]:
df.head()

,payment_channel,amount,date
0,616-1147-19-6,146.00,2022-05-23
1,7238-98788-61-5,71.20,2023-07-06
2,7238-67640-61-5,14.59,2023-09-16
3,7238-127156-61-5,15.55,2023-04-29
4,7238-54956-61-5,21.30,2023-09-04


In [27]:
len(df)

611065

In [28]:
df = df.sort_values(['payment_channel', 'date'])
posid = df.groupby(['payment_channel'])\
            .cumcount().rename('group_position')

df = df.join(posid)

df['amount'] = df['amount'].abs()
#df['date'] = pd.to_datetime(df['date'].astype(str), format='%y%m%d')
df['date'] = pd.to_datetime(df['date'])

print(f"Taking last {100} of each payment_channel...")
df = df.groupby('payment_channel').tail(100)

# Remove payment channels with too few transactions
cnts = df.groupby('payment_channel')['payment_channel'].count().sort_values(ascending=True)
df = df.loc[df['payment_channel'].isin(cnts[cnts > 4].index)]

print("Computing diffdays...")
# compute diffdays
lelele = df.sort_values(['payment_channel', 'date'])\
    .groupby('payment_channel')['date']\
    .diff(1).dt.days\
    .fillna(0)\
    .abs()

df = df.join(lelele.rename('datediff'))

df = df.sort_values(['payment_channel', 'date']).reset_index(drop=True)

df_ori = df.copy()

Taking last 100 of each payment_channel...
Computing diffdays...


### Filtering methods. (See comments for knowing what method is being executed)

In [29]:
%%time
#DBSCAN  ######################################################################
df = dbscanmethod.main_dbscan_method(df_ori, eps_date=.05, eps_amount=.1)
df['cluster_id'] = (df['amount_lvl'].astype(int).astype(str)) + '-' +  df['day_cluster'].astype(int).astype(str)

print(f"Percentage of non-recurrent points using DBSCAN: {len(df[df.is_rec == 0]) / len(df)}")

df.to_parquet('filtered_dbscan.pqt')

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 46238/46238 [01:37<00:00, 471.97it/s]


Percentage of non-recurrent points using DBSCAN: 0.7392646704289466
CPU times: user 1min 40s, sys: 4.01 s, total: 1min 44s
Wall time: 1min 43s


In [30]:
%%time
# GRAPHS  ######################################################################
def flag_matrix(dates, amounts):    
    dates = np.array(dates).astype(np.datetime64)
    amounts = np.array(amounts)
    
    datediffs_to_previous = np.diff(dates).astype('timedelta64[D]')
    datediffs_to_previous = datediffs_to_previous/ np.timedelta64(1, 'D')
    subseries = graphmethod.main_matrix_method_graphs(datediffs_to_previous, amounts, use_dbscan=False)

    flags = np.ones(len(dates)) * -1

    # Return array with subseries ids (eg: [0,1,1,1,1,1,0,0,2,0,2,2,2,2])
    l = list(enumerate(subseries))
    for i, indices in l:
        flags[indices] = i

    return flags

    
def matrix_udf_graphs(data):
    dates = data.date.values
    amounts = data.amount.values
    orders = data.group_position.values
    
    subseries_ids = flag_matrix(dates, amounts)
    
    return pd.Series({'cluster_id': subseries_ids, 'group_position': orders})


bin_nbs = df_ori.sort_values(['payment_channel', 'date'])\
    .groupby(['payment_channel'])\
    .apply(matrix_udf_graphs)\
    .explode(['cluster_id', 'group_position'])

df = pd.merge(df_ori, bin_nbs, on=['payment_channel', 'group_position'])
df['cluster_id'] = df.cluster_id.astype(int)
df['is_rec'] = (df['cluster_id'] >= 0).astype(int)


print(f"Percentage of non-recurrent points using GRAPH: {len(df[df.is_rec == 0]) / len(df)}")

df.to_parquet('filtered_graphs.pqt')

Percentage of non-recurrent points using GRAPH: 0.4457757304431005
CPU times: user 7min 23s, sys: 2.11 s, total: 7min 25s
Wall time: 7min 31s


In [31]:
%%time

# MATRIX  ######################################################################
def matrix_flag(data):
    diff_days = data['datediff'].values[1:]
    #diff_days = data['date'].diff(1).dt.days.dropna().values
    amounts = data['amount'].values
    orders = data.group_position.values
    
    #subseries = split_series_cluster(diff_days, amounts, dates, centroids)

    subseries = matrixmethod.main_matrix_method(diff_days, amounts, use_dbscan=False)
    
    flags = np.ones(len(data)) * -1

    # Return array with subseries ids (eg: [0,1,1,1,1,1,0,0,2,0,2,2,2,2])
    l = list(enumerate(subseries))
    for i, indices in l:
        flags[indices] = i

    return pd.Series({'cluster_id': flags, 'group_position': orders})

# Matrix Method
bin_nbs = df_ori.sort_values(['payment_channel', 'date'])\
    .groupby(['payment_channel'])\
    .apply(matrix_flag)\
    .explode(['cluster_id', 'group_position'])

df = pd.merge(df_ori, bin_nbs, on=['payment_channel', 'group_position'])
df['cluster_id'] = df.cluster_id.astype(int)
df['is_rec'] = (df['cluster_id'] >= 0).astype(int)

print(f"Percentage of non-recurrent points using MATRIX: {len(df[df.is_rec == 0]) / len(df)}")

df.to_parquet('filtered_matrix.pqt')

Percentage of non-recurrent points using MATRIX: 0.6309561150873991
CPU times: user 1min 5s, sys: 517 ms, total: 1min 6s
Wall time: 1min 8s


In [32]:
df

,payment_channel,amount,date,group_position,datediff,cluster_id,is_rec
0,1-106906-29-6,2750.0,2021-12-21,0,0.0,1,1
1,1-106906-29-6,600.0,2022-01-27,1,37.0,1,1
2,1-106906-29-6,600.0,2022-02-24,2,28.0,1,1
3,1-106906-29-6,747.7,2022-03-29,3,33.0,1,1
4,1-106906-29-6,600.0,2022-04-27,4,29.0,1,1
...,...,...,...,...,...,...,...
385754,9998-57780-61-5,55.5,2022-07-02,0,0.0,-1,0
385755,9998-57780-61-5,232.5,2022-09-30,1,90.0,-1,0
385756,9998-57780-61-5,290.5,2023-08-15,2,319.0,-1,0
385757,9998-57780-61-5,115.5,2023-12-01,3,108.0,-1,0


In [33]:
# YOUSFI  ######################################################################

class Config:
    def __init__(self):
        self.type_col='type'
        self.client_col= 'payment_channel'
        self.customer_id= 'payment_channel'
        self.time_col=  'date'
        self.amount_col='amount'
        self.trans_amount='amount'
        self.trans_date=  'date'
        self.trans_type=  'mov_subcat'


config = Config()

def get_fn(df):
    # This is a quick fix, since we don't work with type columns
    df['type'] = 1
    df['mov_subcat'] = 1
    
    dfs =  list((DetectRecurrencyII(
                  trans_data = df.sort_values('date'),
                  client_col= 'payment_channel',
                  time_col=  'date',
                  amount_col='amount',
                  config=config
                  )
           )[1].values())

    # Add a cluster_id col for all dfs
    try:
        dfs = [dfs[i].assign(cluster_id = i).reset_index()  for i, d in enumerate(dfs)]
    except:
        import pdb; pdb.set_trace()


    if len(dfs) > 0:
        concat_df = pd.concat(dfs)
    
        out = pd.merge(
             df, 
             concat_df, 
             left_on=['date', 'amount'], 
             right_on=['date', 'amount'], 
             how='left', indicator=False)

        dfs = out.drop_duplicates(['group_position'], keep='first').fillna(-1)
    else:
        dfs = df.assign(cluster_id = -1)

    return dfs.loc[:, ['date', 'amount', 'cluster_id']]

In [34]:
dft = df_ori.copy()

In [35]:
%%capture
outt = dft.sort_values(['payment_channel', 'date']).groupby('payment_channel').apply(lambda x: get_fn(x))

In [36]:
dft = pd.merge(dft, outt.reset_index().drop('level_1', axis=1), on=('payment_channel', 'date', 'amount')).drop_duplicates(['payment_channel', 'group_position'], keep='first')

In [37]:
dft['cluster_id'] = dft['cluster_id'].fillna(-1).astype(int)

In [38]:
print(f"Percentage of non-recurrent points using YOUSFI: {len(dft[dft.cluster_id < 0]) / len(dft)}")

Percentage of non-recurrent points using YOUSFI: 0.9952483286196823


In [39]:
dft.to_parquet('filtered_yousfi.pqt')

Plot a single example

In [ ]:
t = df.loc[(df.payment_channel == '3514-67514222-PREVOD Z UCTU-PRIJEM')].copy()
t[t.amount > 0]
#t.amount_lvl =  t.amount_lvl.astype(str)


sns.scatterplot(t, 
                x='date', 
                y='amount', 
                style='cluster_id',
                hue='cluster_id'#'cluster_id'
               )

**Recognition stats**

In [41]:
for method in ('dbscan', 'matrix', 'graphs', 'yousfi'):
    df = pd.read_parquet(f'filtered_{method}.pqt')

    if 'is_rec' not in df.columns:
        df['is_rec'] = (df['cluster_id'] >= 0).astype(int)

    total_rec_series = df[df.is_rec > 0].groupby(['payment_channel', 'cluster_id'])['payment_channel'].nunique().sum()
    
    print(f"[{method.upper()}] Percentage of payment channels with some recurrent sub-pattern: {df[df.is_rec > 0].payment_channel.nunique() / df.payment_channel.nunique()} \t Total recurrent subseries: {df[df.is_rec > 0].groupby(['payment_channel', 'cluster_id'])['payment_channel'].nunique().sum()}")

[DBSCAN] Percentage of payment channels with some recurrent sub-pattern: 0.25384132300607076 	 Total recurrent subseries: 13602
[MATRIX] Percentage of payment channels with some recurrent sub-pattern: 0.4376805526481055 	 Total recurrent subseries: 23321
[GRAPHS] Percentage of payment channels with some recurrent sub-pattern: 0.7379945572535064 	 Total recurrent subseries: 42584
[YOUSFI] Percentage of payment channels with some recurrent sub-pattern: 0.008331588863303329 	 Total recurrent subseries: 220
